In [ ]:
!pip install beautifulsoup4

In [1]:
from gensim.models import Word2Vec

In [2]:
class EpochSaver: pass

In [28]:
DATA_HOME = '../../data/w2v'
DATA_HOME = '../../../machine_learning_practico/data/w2v/'

In [ ]:
!mkdir -p $DATA_HOME
!wget https://machine-learning-practico.s3.amazonaws.com/w2v/epoch_10 -O $DATA_HOME/epoch_10
!wget https://machine-learning-practico.s3.amazonaws.com/w2v/epoch_10.trainables.syn1neg.npy -O $DATA_HOME/epoch_10.trainables.syn1neg.npy
!wget https://machine-learning-practico.s3.amazonaws.com/w2v/epoch_10.wv.vectors.npy -O $DATA_HOME/epoch_10.wv.vectors.npy

In [29]:
from pathlib import Path
DATA_HOME = Path(DATA_HOME)

In [44]:
model = Word2Vec.load(str(DATA_HOME / 'epoch_10'))

In [39]:
from IPython.display import HTML, display
from bs4 import BeautifulSoup
import requests
import requests_cache

requests_cache.install_cache('imdb')

def get_name(id):
    response = requests.get(f'https://www.imdb.com/name/{id}/')
    soup = BeautifulSoup(response.content)
    return soup.select('.header .itemprop')[0].text

def get_image(id):
    response = requests.get(f'https://www.imdb.com/name/{id}/')
    soup = BeautifulSoup(response.content)
    candidates = soup.select('#name-poster')
    return candidates[0].attrs['src'] if candidates else 'https://m.media-amazon.com/images/G/01/imdb/images/nopicture/medium/name-2135195744._CB466677935_.png'

def render_person(id):
    name = get_name(id)
    picture = get_image(id)
    return f"""
    <div style="width: 150px; text-align: center">
        <h4 style='margin-top: -5px'>{name}</h4>
        <div style='font-size:75%; margin-bottom: 5px'>{id}</div>
        <a href="https://www.imdb.com/name/{id}" target="_blank">
            <img style="width: 100px; display: block; margin-left: auto; margin-right: auto;" src="{picture}"/>
        </a>
    </div>
    """

In [40]:
def show_similars(id, n=10):
    display(HTML(render_person(id)))
    renders = []
    for similar_id, score in model.wv.most_similar(id, topn=n):
        renders.append(render_person(similar_id))
        
    carousel = ''.join(
        [
            f'<div style="margin-left: 10px; float: left">{p}</div>' 
            for p in renders
        ]
    )
    display(HTML(f'<div style="width: 1800px">{carousel}</div>'))

In [172]:
show_similars('nm0001401')

In [45]:
show_similars('nm0201857')

In [46]:
show_similars('nm0863906')

In [47]:
show_similars('nm0655585')

In [55]:
show_similars('nm0000375')

In [51]:
show_similars('nm0511088')

In [53]:
show_similars('nm0000150')

# Algunos random

In [124]:
import sys
sys.path.append('/Users/przivic/prog/machine_learning_practico')

from lib import data

In [127]:
from pathlib import Path
PATH = Path('../../data/')

In [126]:
import pandas as pd

title_basics = data.load_title_basics(PATH)
principals_df = pd.read_csv(PATH / 'title.principals.tsv', sep='\t')
ratings_df = pd.read_csv(PATH / 'title.ratings.tsv', sep='\t')

/Users/przivic/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [137]:
candidates = (
    principals_df[
        principals_df.category.isin(['actress', 'actor', 'director']) &
        principals_df.tconst.isin(set(title_basics[title_basics.startYear >= 1980].tconst)) &
        principals_df.tconst.isin(set(ratings_df[ratings_df.numVotes > 100000].tconst))
    ].nconst.unique()
)

candidates = [c for c in candidates if c in model.wv.vocab]
len(candidates)

In [177]:
from random import choice

for _ in range(10):
    id = choice(candidates)
    show_similars(id)